## Kapitel 13 - 02

> ### <span style="color:red">ACHTUNG:</span> Bitte zum Starten im Menü `Cell->Run All` ausführen.

## Deaktivieren der Warnungen

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Bienen und Hummeln

#### Listing 13.6

In [ ]:
import pandas as pd
import urllib
import os
import PIL
import shutil
import numpy as np
%matplotlib inline

#### Listing 13.7

In [ ]:
basedir = r'biene_hummel'
dir_dict = {
    'download': os.path.join(basedir,'download'),
    'train':  os.path.join(basedir,'train'),
    'test':  os.path.join(basedir,'test'),
    'valid': os.path.join(basedir,'valid'),
    'train_bee': os.path.join(basedir,'train','biene'),
    'train_bumble': os.path.join(basedir,'train','hummel'),
    'test_bee': os.path.join(basedir,'test','biene'),
    'test_bumble': os.path.join(basedir,'test','hummel'),
    'valid_bee': os.path.join(basedir,'valid','biene'),
    'valid_bumble': os.path.join(basedir,'valid','hummel')}

# Erzeuge die Verzeichnisse (nur wenn sie noch nicht vorhanden sind)
for key in dir_dict:
    if not os.path.exists(dir_dict[key]): 
        os.makedirs(dir_dict[key])

#### Listing 13.8

In [ ]:
url = 'https://raw.githubusercontent.com/dionhagan/naive-bees/master/train_labels.csv'
urllib.request.urlretrieve(url, os.path.join(dir_dict['download'],'train_labels.csv'))


#### Listing 13.9

In [ ]:
df = pd.read_csv(os.path.join(dir_dict['download'],'train_labels.csv'))
df.info()
print(df.head(5))


#### Listing 13.10

In [ ]:
df = df.sort_values(['id'])
df['class'] = np.where(df['genus']==1,'_bumble','_bee')
print(df.head(5))

#### Listing 13.11

In [ ]:

from urllib.request import urlretrieve
for index, row in df.iterrows():
    fname = "{:.0f}.jpg".format(row['id'])
    url = 'https://raw.githubusercontent.com/dionhagan/naive-bees/master/images/train/'+fname
    print(url)   
    urllib.request.urlretrieve(url, os.path.join(dir_dict['download'],fname))


#### Listing 13.12

In [ ]:
from IPython.display import Image
Image(os.path.join(dir_dict['download'],'8.jpg'))

#### Listing 13.13

In [ ]:

for index, row in df.iterrows():
    fname = "{:.0f}.jpg".format(row['id'])
    
    classname = row['class']
    if index<3000:
        dest_dir = os.path.join(dir_dict['train'+classname])
    elif index<3500:
        dest_dir = os.path.join(dir_dict['valid'+classname])
    else:
        dest_dir = os.path.join(dir_dict['test'+classname])
        
    src = os.path.join(dir_dict['download'],fname)
    dest = os.path.join(dest_dir,classname+fname)
    shutil.copyfile(src, dest)


#### Listing 13.14

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

model.summary()


#### Listing 13.15

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(        
        dir_dict['train'],      
        target_size=(200, 200),
        batch_size=20,        
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        dir_dict['valid'],
        target_size=(200, 200),
        batch_size=20,
        class_mode='binary')

#### Listing 13.16

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

print(class_weights)

#### Listing 13.17

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      class_weight = class_weights)

#### Listing 13.18

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Trainingsgenauigkeit')
plt.plot(epochs, val_acc, 'b', label='Validierungsgenauigkeit')
plt.title('Trainings- und Validierungsgenauigkeit')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Trainings Loss')
plt.plot(epochs, val_loss, 'b', label='Validierungs loss')
plt.title('Trainings und Validierungs Loss')
plt.legend()

plt.show()

#### Zusatzlisting für die Prädiktion

In [ ]:
validation_generator.reset()
prediction = model.predict_generator(validation_generator, verbose=1)

predicted_class_indices = np.argmax(prediction,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

#decoded = decode_predictions(prediction, top=3)[0]
